In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_515820/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 16:25:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 16:25:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Products table
products_data = {
    'product_id': [1, 2],
    'price': [100, 200]
}

# Sample data for Purchases table
purchases_data = {
    'invoice_id': [1, 3, 2, 2, 4],
    'product_id': [1, 2, 2, 1, 1],
    'quantity': [2, 1, 3, 4, 10]
}

# Create DataFrame for Products
products_df = pd.DataFrame(products_data)

# Create DataFrame for Purchases
purchases_df = pd.DataFrame(purchases_data)

# Display the DataFrames
print("Products table:")
print(products_df)
print("\nPurchases table:")
print(purchases_df)



df_person = spark.createDataFrame(products_df)
df_person.createOrReplaceTempView("Products")

df_person = spark.createDataFrame(purchases_df)
df_person.createOrReplaceTempView("Purchases")



Products table:
   product_id  price
0           1    100
1           2    200

Purchases table:
   invoice_id  product_id  quantity
0           1           1         2
1           3           2         1
2           2           2         3
3           2           1         4
4           4           1        10


In [8]:
query = """
    with t as (
        select
            pu.*,
            pr.price * pu.quantity as total_price
        from Purchases pu left join Products pr on pu.product_id = pr.product_id 
    )

    select
        t.product_id,
        t.quantity,
        t.total_price
    from (
        select
            invoice_id,
            sum(total_price) as total_price
        from t 
        group by invoice_id
        order by total_price desc, invoice_id 
        limit 1
    ) as k left join t on k.invoice_id = t.invoice_id

"""

In [9]:
spark.sql(query).show()

+----------+--------+-----------+
|product_id|quantity|total_price|
+----------+--------+-----------+
|         2|       3|        600|
|         1|       4|        400|
+----------+--------+-----------+

